In [43]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
import tensorflow
tensorflow.keras.__version__
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
from sklearn import preprocessing

In [44]:
data = pd.read_csv('../Resources/cumulative.csv')
data = data.fillna(0)
data.head()
X = data
X.head()

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,0,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,0,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [45]:
from sklearn.datasets import make_classification            
X = data.drop("kepoi_name",  axis=1)
X = X.drop("kepler_name",  axis=1)
X = X.drop("koi_disposition",  axis=1)
X = X.drop("koi_pdisposition",  axis=1)
X = X.drop("koi_tce_delivname", axis=1)
X = X.drop("kepid", axis=1)
X = X.fillna(0)

y = data["kepler_name"]
y = y.astype(str)
#y = y.values

#y = y.reshape(-1, 1)

print(X.shape, y.shape)


(9564, 44) (9564,)


In [46]:
#Creating Labels
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)
y_categorical = to_categorical(encoded_y)

In [47]:
#Splitting our training data and test data
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, random_state=100)
X_train = X_train.values
X_test = X_test.values

In [48]:
#Scaling our data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [49]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=y_categorical.shape[1], activation='softmax'))

In [50]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(X_train_scaled.shape)
print(y_train.shape)

(7173, 44)
(7173, 2295)


In [51]:
#Running the model
model.fit(
    X_train_scaled,
    y_train,
    epochs=60,
    shuffle=True,
    verbose=1
)

Epoch 1/60
225/225 [==============================] - 0s 1ms/step - loss: 3.0347 - accuracy: 0.7399
Epoch 2/60
225/225 [==============================] - 0s 1ms/step - loss: 2.1087 - accuracy: 0.7613
Epoch 3/60
225/225 [==============================] - 0s 1ms/step - loss: 2.0748 - accuracy: 0.7613
Epoch 4/60
225/225 [==============================] - 0s 1ms/step - loss: 2.0598 - accuracy: 0.7613
Epoch 5/60
225/225 [==============================] - 0s 1ms/step - loss: 2.0535 - accuracy: 0.7613
Epoch 6/60
225/225 [==============================] - 0s 1ms/step - loss: 2.0447 - accuracy: 0.7613
Epoch 7/60
225/225 [==============================] - 0s 1ms/step - loss: 2.0309 - accuracy: 0.7613
Epoch 8/60
225/225 [==============================] - 0s 1ms/step - loss: 2.0156 - accuracy: 0.7613
Epoch 9/60
225/225 [==============================] - 0s 1ms/step - loss: 1.9972 - accuracy: 0.7613
Epoch 10/60
225/225 [==============================] - 0s 1ms/step - loss: 1.9592 - accuracy: 0.7613

In [54]:
#Prediction Percentage
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

75/75 - 0s - loss: 9.2179 - accuracy: 0.7344
Normal Neural Network - Loss: 9.217897415161133, Accuracy: 0.7344207167625427


In [ ]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [55]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
